# Limpieza y preprocesamiento de datos

## Importación librerias

In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from googletrans import Translator
import pycountry
import requests
from tzlocal import get_localzone
import pytz
from datetime import datetime
import re
import emoji
from nltk.sentiment.vader import SentimentIntensityAnalyzer

## Lectura de datos

In [2]:
tweets_guerra = pd.read_excel("tweets_guerra.xlsx")

In [3]:
tweets_guerra.head()

,id,usuario,tweet,ubicacion,nombre_de_pila,fecha
0,1751724377383550976,laprensa,Doce miembros de la agencia para los refugiado...,Nicaragua,LA PRENSA Nicaragua,2024-01-28 21:50:11+00:00
1,1751723773194027008,AgenciaAJN,El gran rabino de Rusia se expresó en público ...,Argentina,Agencia AJN,2024-01-28 21:47:47+00:00
2,1751721984226959872,helenezonana,Que liberen a los rehenes YA! Basta de tanta b...,".,.",Helene Zonana,2024-01-28 21:40:41+00:00
3,1751719317958172928,LawAndTrends,La CIJ exige a Israel impedir un genocidio con...,the world,LawAndTrends💙💛,2024-01-28 21:30:05+00:00
4,1751718215938458112,Alexand22956267,Querem discutir o cessar-fogo sem a participaç...,NaN,Alexandre Moreira,2024-01-28 21:25:42+00:00


In [4]:
tweets_guerra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              2000 non-null   int64 
 1   usuario         2000 non-null   object
 2   tweet           2000 non-null   object
 3   ubicacion       1413 non-null   object
 4   nombre_de_pila  2000 non-null   object
 5   fecha           2000 non-null   object
dtypes: int64(1), object(5)
memory usage: 93.9+ KB


In [5]:
# Verificación de NA
tweets_guerra.isna().sum()

id                  0
usuario             0
tweet               0
ubicacion         587
nombre_de_pila      0
fecha               0
dtype: int64

In [6]:
# Cambiar NA de la columna ubicacion por "sin ubicación"
tweets_guerra['ubicacion'] = tweets_guerra['ubicacion'].fillna('sin ubicación')

In [7]:
# Verificar inexistencia de NA
tweets_guerra.isna().sum()

id                0
usuario           0
tweet             0
ubicacion         0
nombre_de_pila    0
fecha             0
dtype: int64

## Cambiar ubicación por país

In [8]:
# Función que utiliza librería geopy para obtener el pais de la ubicación proporcionada.
# fuente: https://pypi.org/project/geopy/
def obtener_pais_ubi(ubicacion):
    geolocalizador = Nominatim(user_agent="localizacion")
    try:
        location = geolocalizador.geocode(ubicacion)
        if location:
            pais = location.address.split(",")[-1].strip()
        else:
            pais = 'sin ubicación'
        return {ubicacion: pais}
    except Exception as e:
        print(f"Error al obtener país para '{ubicacion}': {e}")
        return {ubicacion: 'sin ubicación'}


In [9]:
diccionario_paises = {}
tweets_ubicaciones = tweets_guerra["ubicacion"].unique()

for ubicacion in tweets_ubicaciones:
    diccionario_paises.update(obtener_pais_ubi(ubicacion))

Error al obtener país para 'Ciudad Autónoma de Buenos Aire': HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Ciudad+Aut%C3%B3noma+de+Buenos+Aire&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Error al obtener país para 'Panamá. Panamá': HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Panam%C3%A1.+Panam%C3%A1&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Error al obtener país para 'Provincia de Bs. Aires': HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Provincia+de+Bs.+Aires&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed ou

In [10]:
# Arreglar manual ubicaciones no encontradas en la función 
# ** Ajustar u omitir esta celda para reproducir **
diccionario_paises['Ciudad Autónoma de Buenos Aire']= 'Argentina'
diccionario_paises['Provincia de Bs. Aires']= 'Argentina'
diccionario_paises['the world'] = 'sin ubicación'
diccionario_paises['Mundo'] = 'sin ubicación'
diccionario_paises['worldwide'] = 'sin ubicación'
diccionario_paises['en mi país, Venezuela'] = 'Venezuela'
diccionario_paises['En el Sector de lavapie SC '] = 'sin ubicación'
diccionario_paises['the word'] = 'sin ubicación'
diccionario_paises['Cidade do Vaticano'] = "El vaticano"
diccionario_paises['República Socialista '] = 'sin ubicación'
diccionario_paises['República Popular Democrática '] = 'sin ubicación'
diccionario_paises['Estambul'] = 'Turquía'
diccionario_paises['Panamá. Panamá']= 'Panamá'
diccionario_paises['Panamá/ Panama']= 'Panamá'
diccionario_paises['E #454 esq. 19 La Habana, Cuba'] = "Cuba"
diccionario_paises['Porto, Norte de Portugal, UE'] = "Portugal"
diccionario_paises['Rep. Bolivariana de Venezuela']= "Venezuela"
diccionario_paises['In and Around Boston']="Estados Unidos"
diccionario_paises['Panamá, Rep. de Panamá']="Panamá"
diccionario_paises['CDMX / Guadalajara / Monterrey']="México"
diccionario_paises['3426 Ashlock Dr Houston,TX 770']="Estados Unidos"
diccionario_paises['De Galiza a Caracas, Venezuela']="Venezuela"
diccionario_paises['Barranquilla, Tel 6053735050']="Colombia"
diccionario_paises['  🇪🇸 ']="España"
diccionario_paises['Argentina, 🕊️⚠️ mod/they/them']="Argentina"
diccionario_paises['Barcelona República Catalana']="España"
diccionario_paises['Israel']="Israel"
diccionario_paises['Paraguay / Paraguái']="Paraguay"
diccionario_paises['الأراضي الفلسطينية']="Palestina"
diccionario_paises['Papua Niugini']="Nueva Guinea"
diccionario_paises['Schweiz/Suisse/Svizzera/Svizra']="Suiza"
diccionario_paises['لبنان']= "Líbano"
diccionario_paises['Éire / Ireland'] = "Irlanda"
diccionario_paises['België / Belgique / Belgien']="Bélgica"
diccionario_paises['中国']= "China"
diccionario_paises['Россия']= "Rusia"
diccionario_paises['السودان']= "Sudan"
diccionario_paises['قطر']= "Qatar"
diccionario_paises['قطر']= "Qatar"
diccionario_paises['Norge']= "Noruega"
diccionario_paises['Deutschland']= "Alemania"
diccionario_paises['Nederland']= "Paises Bajos"
diccionario_paises['عمان']= "Oman"
diccionario_paises['Magyarország']= "Hungría"
diccionario_paises['Монгол улс ᠮᠤᠩᠭᠤᠯ ᠤᠯᠤᠰ']= "Mongolia"
diccionario_paises['العراق']= "Irak"
diccionario_paises['Shqipëria']= "Albania"
diccionario_paises['ایران']= "Iran"

In [11]:
tweets_guerra['ubicacion'] = tweets_guerra['ubicacion'].replace(diccionario_paises)

## Traducir país y tweet a español

In [12]:
# Función que utiliza la librería googletrans para detectar el idioma en el cual está escrito el texto de entrada y lo traduce a español.
# Fuente: https://pypi.org/project/googletrans/
def traducir_a_espanol(texto):
    try:
        translator = Translator()
        traduccion = translator.translate(texto, src='auto', dest='es')
        return traduccion.text
    except Exception as e:
        print(f"Error: {e} al traducir el tweet: {texto}")
        return texto 

In [13]:
# Traducir ubicaciones (paises)
diccionario_paises_traducido = {clave: traducir_a_espanol(valor) for clave, valor in diccionario_paises.items()}

Error: The read operation timed out al traducir el tweet: Россия
Error: 'NoneType' object is not iterable al traducir el tweet: Moldova


In [14]:
# Arreglar manual ubicaciones mal traducidas por la función 
# ** Ajustar u omitir esta celda para reproducir **
for clave, valor in diccionario_paises_traducido.items():
    if (valor == 'Espacio'):
        diccionario_paises_traducido[clave] = "España"
for clave, valor in diccionario_paises_traducido.items():
    if (valor == 'Porcelana'):
        diccionario_paises_traducido[clave] = "China"
for clave, valor in diccionario_paises_traducido.items():
    if (valor == 'Lo más conocido ᠮᠤᠩᠭᠤᠯ ᠤᠯᠤᠰ'):
        diccionario_paises_traducido[clave] = "Mongolia"
for clave, valor in diccionario_paises_traducido.items():
    if (valor == 'Guía'):
        diccionario_paises_traducido[clave] = "sin ubicación"

In [15]:
tweets_guerra['ubicacion']  = tweets_guerra['ubicacion'].replace(diccionario_paises_traducido)

Antes de traducir tweet se procederá a limpiarlo.


In [16]:
# Función que elimina @, # y emojis de los tweets.
def limpiar_tweet(tweet):
    tweet_sin_menciones_hashtags = re.sub(r'[@#]\S+', '', tweet)
    tweet_limpiado  = re.sub(r'\s?[\U00010000-\U0010ffff]\s?', '', tweet_sin_menciones_hashtags)
    return tweet_limpiado

tweets_guerra["tweet"] = tweets_guerra["tweet"].apply(limpiar_tweet)

In [17]:
#Traducir tweets
tweets_guerra["tweet"] = tweets_guerra["tweet"].apply(traducir_a_espanol)

Error: list index out of range al traducir el tweet: https://t.co/G80gyYqBi6
Error: list index out of range al traducir el tweet: https://t.co/kEO7gnUOI3
Error: the JSON object must be str, bytes or bytearray, not NoneType al traducir el tweet: El olivo, símbolo de Palestina y víctima muda de la guerra de Israel en Gaza
La pérdida de estos firmes compañeros ha dejado profundas cicatrices en los corazones de muchos palestinos de Gaza.
Por Ruwaida Amer  English Al Jazeera
Al-Fukhari, Gaza – Ahlam Saqr, de 50 años, lloró la mañana en que sus hijos comenzaron a cortar ramas de sus olivos para quemarlas y hacer fuego para cocinar, mantenerse caliente y calentar agua para bañarse.

Era una cuestión de supervivencia, dice, permitir que la familia sobreviviera al implacable bombardeo israelí de Gaza. Pero eso no hizo que fuera más fácil ver cómo desarmaban sus cuatro amados árboles.

Obligado a perder 'compañeros de vida'
Gaza se encuentra bajo un brutal bombardeo y asedio israelí que ha despl

## Obtener codigo iso-2 del país

In [18]:
# Función que utiliza la librería pycountry para encontrar codigo iso-2 del país.
# Fuente: https://pypi.org/project/pycountry/
def obtener_codigos_paises(nombres_paises):
    codigos_paises = {}
    
    for nombre_pais in nombres_paises:
        try:
            resultado_busqueda = pycountry.countries.search_fuzzy(nombre_pais)
            pais = resultado_busqueda[0].alpha_2
            codigos_paises[nombre_pais] = pais
        except LookupError:
            try:
                translator = Translator()
                pais_en = translator.translate(nombre_pais, src='es', dest='en')
                pais = pycountry.countries.search_fuzzy(pais_en.text)[0]
                codigos_paises[nombre_pais] = pais.alpha_2
            except LookupError:
                codigos_paises[nombre_pais] = "no se encontró código"

    return codigos_paises


In [19]:
lista_paises = tweets_guerra[tweets_guerra['ubicacion'] != "sin ubicación"]["ubicacion"].unique()
codigos = obtener_codigos_paises(lista_paises)

diccionario_paises_codigos = {pais: codigo for pais, codigo in codigos.items()}

SubdivisionHierarchy(code='GT-GU', country_code='GT', name='Guatemala', parent_code=None, type='Department')
SubdivisionHierarchy(code='MX-MEX', country_code='MX', name='México', parent_code=None, type='State')
SubdivisionHierarchy(code='PA-8', country_code='PA', name='Panamá', parent_code=None, type='Province')
SubdivisionHierarchy(code='GN-ML', country_code='GN', name='Mali', parent='L', parent_code='GN-L', type='Prefecture')


In [20]:
# Arreglar manual codigos no encontrados por la función 
# ** Ajustar u omitir esta celda para reproducir **
diccionario_paises_codigos["El vaticano"] = "VA"

In [21]:
tweets_guerra["codigo_pais"]= tweets_guerra["ubicacion"].replace(diccionario_paises_codigos)

## Ajustar hora según ubicación 

In [22]:
# Función que utiliza la librería pytz para cambiar la zona horaria según la ubicación.
# Fuente: https://pypi.org/project/pytz/
def ajustar_hora(row):
    pais = row['codigo_pais']
    hora = row['fecha']
    try:
        zona_horaria = pytz.country_timezones[pais][0]
    except KeyError:
        zona_horaria = pytz.country_timezones["ES"][0]
    
    dt = datetime.strptime(hora, '%Y-%m-%d %H:%M:%S%z')
    dt = dt.astimezone(pytz.timezone(zona_horaria))

    return dt.strftime('%Y-%m-%d %H:%M:%S')

In [23]:
tweets_guerra["hora_pais"]=tweets_guerra.apply(ajustar_hora, axis=1)

## Organizar fechas y horas

In [24]:
tweets_guerra.rename(columns={'fecha': 'fecha_hora'}, inplace=True)
tweets_guerra["fecha_hora"]= pd.to_datetime(tweets_guerra["fecha_hora"])
tweets_guerra["fecha"] = tweets_guerra["fecha_hora"].dt.date
tweets_guerra["fecha"]= pd.to_datetime(tweets_guerra["fecha"])
tweets_guerra["hora"] = tweets_guerra["fecha_hora"].dt.time
tweets_guerra = tweets_guerra.drop('fecha_hora', axis=1)

In [25]:
meses = {1: 'enero', 2: 'febrero', 3: 'marzo', 4: 'abril', 5: 'mayo', 6: 'junio', 7: 'julio', 8: 'agosto', 9: 'septiembre', 10: 'octubre', 11: 'noviembre', 12: 'diciembre'}
dias = {'Monday': 'lunes', 'Tuesday': 'martes', 'Wednesday': 'miércoles', 'Thursday': 'jueves', 'Friday': 'viernes', 'Saturday': 'sábado', 'Sunday': 'domingo'}

tweets_guerra['mes'] = tweets_guerra['fecha'].dt.month.map(meses)
tweets_guerra['año'] = tweets_guerra['fecha'].dt.year
tweets_guerra['dia'] = tweets_guerra['fecha'].dt.day
tweets_guerra['dia_semana'] = tweets_guerra['fecha'].dt.day_name().map(dias)

In [26]:
tweets_guerra["hora"] = pd.to_datetime(tweets_guerra["hora"], format='%H:%M:%S')
limites = [0, 5, 12, 18, 24]
etiquetas = ['Noche', 'Mañana', 'Tarde', 'Noche']
tweets_guerra['periodo'] = pd.cut(tweets_guerra['hora'].dt.hour, bins=limites, labels=etiquetas, right=False,ordered=False)

## Obtener coordenadas de la ubicación

In [27]:
# Función que utiliza la librería requests para hacer un web scrapping y
# obtener la latitud y longitud de países a partir de códigos ISO-2. 
# Fuente : https://pypi.org/project/requests/
def obtener_coordenadas_iso2(codigos_iso2):
        resultados = []

        for codigo_iso2 in codigos_iso2:
            try:
                    url = f"https://restcountries.com/v2/alpha/{codigo_iso2}"
                    respuesta = requests.get(url)
                    
                    if respuesta.status_code == 200:
                        datos_pais = respuesta.json()
                        longitud = datos_pais['latlng'][1]
                        latitud = datos_pais['latlng'][0]
                        
                        resultados.append({
                            'codigo_pais': codigo_iso2,
                            'Latitud': latitud,
                            'Longitud': longitud
                        })
            except Exception as e:
                print(e)
        df = pd.DataFrame(resultados)
        return df

In [28]:
data_coordenadas= obtener_coordenadas_iso2(tweets_guerra.codigo_pais)

In [29]:
tweets_guerra = tweets_guerra.merge(data_coordenadas, on= "codigo_pais",how="left").drop_duplicates().reset_index()


## Análisis de sentimientos

In [30]:
# Función que utiliza la librería nltk para detectar si el tweet es positivo, negativo o neutro.
# Fuente: https://www.nltk.org/api/nltk.sentiment.vader.html
def analisis_sentimientos(texto):
    analizador = SentimentIntensityAnalyzer()

    sentiment_score = analizador.polarity_scores(texto)

    if sentiment_score['compound'] >= 0.05:
        return "Positivo"
    elif sentiment_score['compound'] <= -0.05:
        return "Negativo"
    else:
        return "Neutral"

In [31]:
tweets_guerra["sentimiento"]=tweets_guerra['tweet'].apply(analisis_sentimientos)

In [32]:
tweets_guerra.to_csv("tweets_guerra_limpio.csv")